In [1]:
%pip install osmium folium memory_profiler

Note: you may need to restart the kernel to use updated packages.


In [2]:
import vrp
import geographs

In [3]:
Depot_coord = (18.452521, -69.921330)
qty_clients = 30
cap_trucks = 80

In [4]:
print("Loading data... ")

geoGraph = geographs.GeoGraph(limit_coords=Depot_coord)
print("Data loaded! ")

Loading data... 
Available nodes before condition: 133161
Available nodes after condition: 12730
Data loaded! 


In [5]:
Depot_node, clients, qty_trucks = vrp.create_nodes(geoGraph, Depot_coord, qty_clients, cap_trucks)

print("Depot node: ", Depot_node)
print("qty of trucks: ", qty_trucks)

Depot node:  GeoNode 9244572760 at pos (-699214186, 184525750), coords (18.452575, -69.9214186)
qty of trucks:  4


In [6]:
qty_poblacion = 200
n_elite = 10
n_generations = 50
prob_de_mut = 0.5

solution = vrp.create_clusters(geoGraph, qty_clients, qty_trucks, cap_trucks, qty_poblacion, n_elite, n_generations, clients, prob_de_mut)

print("solution: ", solution)

solution:  [3, 0, 1, 0, 2, 1, 0, 0, 1, 0, 1, 3, 3, 2, 0, 1, 1, 2, 2, 0, 2, 3, 2, 1, 2, 1, 3, 0, 2, 3]


In [7]:
n_nearest_nodes = 3

routes, paths = vrp.create_routes(geoGraph, Depot_node, clients, qty_trucks, solution, n_nearest_nodes)

print("routes: ", routes)

for graph in routes:
    print(graph.get_nodes())


Truck 1/4
Len del client cluster: 8
Nearest nodes for client 308801172: [6689629528, 308908431, 308924011, 9244572760]
Connecting 308801172 and 6689629528
Connecting 308801172 and 308908431
Connecting 308801172 and 308924011
Connecting 308801172 and 9244572760
Nearest nodes for client 6689629528: [308801172, 308908431, 308924011, 9244572760]
Connecting 6689629528 and 308801172
Connecting 6689629528 and 308908431
Connecting 6689629528 and 308924011
Connecting 6689629528 and 9244572760
Nearest nodes for client 308924011: [308908431, 308801172, 6689629528, 9244572760]
Connecting 308924011 and 308908431
Connecting 308924011 and 308801172
Connecting 308924011 and 6689629528
Connecting 308924011 and 9244572760
Nearest nodes for client 8349937358: [4591901877, 4596989378, 310493754, 9244572760]
Connecting 8349937358 and 4591901877
Connecting 8349937358 and 4596989378
Connecting 8349937358 and 310493754
Connecting 8349937358 and 9244572760
Nearest nodes for client 308908431: [308801172, 308924

In [8]:
Depot_node.name

9244572760

In [9]:
import folium
colors = ['red', 'blue', 'green', 'purple']
depot_color = 'black'

map = folium.Map(location=Depot_coord, zoom_start=17)

i = 0
for route in paths:
    for initial_node_name in route:
        for goal_node_name in route[initial_node_name]:
            
            initial_node = routes[i].get_node_by_name(initial_node_name)
            goal_node = routes[i].get_node_by_name(goal_node_name)

            initial_marker = [initial_node.lat, initial_node.lon]
            folium.Marker(initial_marker,popup=initial_node.name, icon=folium.Icon(color=colors[i])).add_to(map)
            
            goal_marker = [goal_node.lat, goal_node.lon]
            folium.Marker(goal_marker,popup=goal_node.name, icon=folium.Icon(color=colors[i])).add_to(map)

            path = route[initial_node_name][goal_node_name]
            points = []

            for node in path:
                point = [node.lat, node.lon]
                points.append(point)

            folium.PolyLine(points, color=colors[i], weight=5, opacity=1).add_to(map)
    i+=1

depot_marker = [Depot_node.lat, Depot_node.lon]
folium.Marker(depot_marker,popup=Depot_node.name, icon=folium.Icon(color=depot_color)).add_to(map)

map

In [11]:
import genetic_algorithms_vin as gav
import random

answers = []

for route in routes:

    graph = route.get_adj_matrix(Depot_node)

    #Params
    num_gen = 5000
    population_num = 200
    solutions_to_choose = 5
    mutation_prob = 0.9

    cities = list(range(0,len(graph)))

    initial_city = 0

    #initial population
    solutions = gav.create_population(population_num, cities, initial_city)
    #print('Propulation: ', solutions)

    best_solutions_by_gen = []

    #For each generation
    for gen in range(num_gen):

        #Calculate the total distance of each solution
        distances = gav.calc_distances_paths(graph, initial_city, solutions)
        #print("distances: ", distances)
        
        #Calculate the fitness for each solution
        fitnesess = gav.calc_fitness(distances)
        #print("fitnesess: ", fitnesess)

        if(sum(fitnesess) == 0):
            choosen_solutions = solutions[:solutions_to_choose]
        else:
            #Choose k solutions randomly based on their fitnesess
            choosen_solutions = random.choices(solutions, weights=fitnesess, k=solutions_to_choose)
        #print("choosen solutions", choosen_solutions)

        #Save the best k solutions of this generation
        best_solutions_by_gen.append(choosen_solutions)

        #Use best solutions to create a new generation by croosover and mutation (based in mutation_prob)
        crossovered_solutions = gav.crossover_solutions(choosen_solutions, population_num)
        #print("Crossovered_solutions: ", crossovered_solutions)
        solutions = gav.mutate_solutions(crossovered_solutions, mutation_prob)
        #print("Mutated_solutions: ", solutions)

    best_solution_by_gen = []

    for gen_solutions in best_solutions_by_gen:
        distances = gav.calc_distances_paths(graph, initial_city, gen_solutions)
        best_solution_of_gen = gav.sort_solutions_by_distance(gen_solutions, distances)[-1]
        best_solution_by_gen.append(best_solution_of_gen)

    final_answer, cost = sorted(best_solution_by_gen,key=lambda x: x[1])[0]

    final_answer.append(initial_city)

    print("Answer: ", final_answer, cost)

    answers.append(final_answer)

Answer:  [0, 6, 2, 7, 1, 5, 3, 8, 4, 0] inf
Answer:  [0, 1, 6, 8, 2, 3, 7, 5, 4, 0] 24.553736749819794
Answer:  [0, 6, 5, 1, 2, 8, 7, 3, 4, 0] 15.817742580851712
Answer:  [0, 2, 6, 5, 4, 3, 1, 0] 16.9180015937916


In [21]:
answers

[[0, 7, 6, 4, 5, 8, 1, 2, 3, 0],
 [0, 7, 5, 2, 3, 4, 8, 6, 1, 0],
 [0, 3, 7, 2, 6, 5, 4, 8, 1, 0],
 [0, 5, 2, 3, 6, 4, 1, 0]]

In [13]:
graph[0]

[inf,
 5.777239125871959,
 5.639224268027055,
 5.10116665601159,
 6.081321487966189,
 1.208225349276105,
 2.435193901615983,
 1.0762593667425546,
 7.841347650539463]

In [11]:
#import swarm_algorithms as sa
import genetic_algorithms as ga

answers = []

for route in routes:

    graph_adj_matrix = route.get_adj_matrix(Depot_node)

    initial_city = 0 #i

    aco = ga.GeneticAlgorithm(cities = graph_adj_matrix, start_city = initial_city)
    aco_solution, aco_cost  = aco.solve()
    print("Answer: ", aco_solution)
    answers.append(aco_solution)

Answer:  [8, 6, 4, 1, 3, 2, 7, 5, 0]
Answer:  [2, 6, 8, 3, 7, 5, 1, 4, 0]
Answer:  [2, 3, 5, 4, 6, 8, 1, 7, 0]
Answer:  [3, 5, 1, 4, 2, 6, 0]


In [12]:
for answer in answers:
    answer.insert(0, 0)

In [13]:
answers

[[0, 8, 6, 4, 1, 3, 2, 7, 5, 0],
 [0, 2, 6, 8, 3, 7, 5, 1, 4, 0],
 [0, 2, 3, 5, 4, 6, 8, 1, 7, 0],
 [0, 3, 5, 1, 4, 2, 6, 0]]

In [14]:
import folium
colors = ['red', 'blue', 'green', 'purple']
depot_color = 'black'

map = folium.Map(location=Depot_coord, zoom_start=17)

i = 0
for route_graph,route, answer in zip(routes,paths, answers):
    
    nodes_answers = [route_graph.index_to_node[answer_node] for answer_node in answer]

    print("Nodes answers: ", nodes_answers)

    j = 0
    for j in range(len(nodes_answers)-1):
        initial_node = routes[i].get_node_by_name(nodes_answers[j])
        goal_node = routes[i].get_node_by_name(nodes_answers[j+1])

        initial_marker = [initial_node.lat, initial_node.lon]
        folium.Marker(initial_marker,popup=initial_node.name, icon=folium.Icon(color=colors[i])).add_to(map)
        
        goal_marker = [goal_node.lat, goal_node.lon]
        folium.Marker(goal_marker,popup=goal_node.name, icon=folium.Icon(color=colors[i])).add_to(map)

        print("Initial node: ", initial_node)
        print("goal node: ", goal_node)

        path = route[initial_node.name][goal_node.name]
        points = []

        for node in path:
            point = [node.lat, node.lon]
            points.append(point)

        folium.PolyLine(points, color=colors[i], weight=5, opacity=1).add_to(map)
    i+=1

depot_marker = [Depot_node.lat, Depot_node.lon]
folium.Marker(depot_marker,popup=Depot_node.name, icon=folium.Icon(color=depot_color)).add_to(map)

map

Nodes answers:  [309456309, 9244572774, 310290253, 309456600, 309426807, 6241097361, 309434825, 309216415, 309216435, 309456309]
Initial node:  GeoNode 309456309 at pos (-699216191, 184523209), coords (18.4523209, -69.9216191)
goal node:  GeoNode 9244572774 at pos (-699196002, 184535205), coords (18.4535205, -69.9196002)
Initial node:  GeoNode 9244572774 at pos (-699196002, 184535205), coords (18.4535205, -69.9196002)
goal node:  GeoNode 310290253 at pos (-699216124, 184610266), coords (18.4610266, -69.9216124)
Initial node:  GeoNode 310290253 at pos (-699216124, 184610266), coords (18.4610266, -69.9216124)
goal node:  GeoNode 309456600 at pos (-699185583, 184539754), coords (18.4539754, -69.9185583)
Initial node:  GeoNode 309456600 at pos (-699185583, 184539754), coords (18.4539754, -69.9185583)
goal node:  GeoNode 309426807 at pos (-699680050, 184497904), coords (18.4497904, -69.968005)


KeyError: 309426807

In [15]:
paths[0][309456600].keys()

dict_keys([9244572774, 6241097361, 310290253, 309456309])

In [81]:
Depot_node.name

9244572760

In [73]:
answers

[[0, 2, 4, 8, 6, 7, 1, 3, 5, 0],
 [0, 1, 5, 7, 8, 4, 6, 3, 2, 0],
 [0, 6, 8, 7, 2, 3, 5, 1, 4, 0],
 [0, 4, 6, 3, 1, 5, 2, 0]]

In [70]:
paths[0].keys()

dict_keys([7964870505, 308816334, 8238995479, 4607510935, 309673864, 4586441234, 1674101449, 308953111, 9244572760])

In [61]:
nodes_answers = [routes[1].index_to_node[answer_node] for answer_node in aco_solution]
nodes_answers

[9244572760,
 310574372,
 1666178787,
 4586098849,
 310762690,
 1867361121,
 310773074,
 310890369,
 1674103193,
 9244572760]

In [55]:
paths[1][310762690][9244572760]

In [39]:
answer_path = [paths[1][answer_node] for answer_node in nodes_answers][0]


In [40]:
answer_path.keys()

dict_keys([310773074, 1666178787, 310762690, 9244572760, 1674103193, 310574372, 310890369, 4586098849, 1867361121])

In [15]:

import numpy as np
from ACO import Colony

inf = float('inf')

answers = []

for route in routes:

  answer = None

  graph_adj_matrix = route.get_adj_matrix(Depot_node)

  initial_city = 0 #i
  num_ants = 10
  num_iterations = 100

  #Params
  desirability_exp = 0.7

  pheromone_Initial_Intensity = 1.0
  pheromone_Intensity = 1
  pheromone_exp = 1.1
  pheromone_evaporation_rate = 0.2

  #Pheromoes numpy array as an adjencency matrix (like graph)
  pheromones_traces = []
  for city in graph_adj_matrix:
    pheromones_traces.append([pheromone_Initial_Intensity]*len(city))

  pheromones_traces = np.array(pheromones_traces)

  print(pheromones_traces.shape)

  colony = Colony(Graph=graph_adj_matrix,ants_num=num_ants, pheromones_traces=pheromones_traces, params=[desirability_exp, pheromone_Initial_Intensity, pheromone_Intensity, pheromone_exp, pheromone_evaporation_rate])

  answer = colony.solve(Original_graph=graph_adj_matrix,initial_city=initial_city, iterations=num_iterations)
  print("Answer: ", answer)

  answers.append(answer)


(9, 9)
Answer:  [0, 3, 1, 2, 7, 5]
(9, 9)
Answer:  [0, 6, 4, 1, 7]
(9, 9)
Answer:  [0, 7, 8, 5, 6]
(7, 7)
Answer:  [0, 1, 4, 2, 3]


In [21]:
for node in answer:
    print(routes[1].index_to_node[node])

9244572760
1666178787
1867361121
310773074
310762690
